In [1]:
import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect(database='liang-bo.wang_project1')

In [3]:
%%time
df = pd.read_sql_query('''
WITH trips AS (
    SELECT pickup_datetime AT TIME ZONE 'EDT' AS pickup_datetime,
           dropoff_datetime AT TIME ZONE 'EDT' AS dropoff_datetime, 
           EXTRACT(EPOCH FROM dropoff_datetime - pickup_datetime) AS trip_duration,
           trip_distance, 
           total_amount
    FROM taxi_trips
    WHERE 
        ST_DWithin(
            pickup_loc, 
            ST_Transform(ST_SetSRID(ST_MakePoint('-73.986166', '40.758585'), 4326), 3627),
            800
        )
        AND ST_Within(dropoff_loc, (SELECT geom FROM taxi_zones WHERE gid = 132))
    LIMIT 1000
) 
SELECT 
    pickup_datetime, dropoff_datetime, trip_duration, trip_distance, total_amount,
    max(one_hour_acc_precipitation) AS max_precipitation,
    avg(temperature) AS avg_temperature
FROM trips
LEFT JOIN weather_normalize
ON date_trunc('hour', record_datetime) BETWEEN
    date_trunc('hour', pickup_datetime AT TIME ZONE 'EDT') AND date_trunc('hour', dropoff_datetime AT TIME ZONE 'EDT') 
    AND one_hour_acc_precipitation != double precision 'NaN' 
    AND temperature != double precision 'NaN'
GROUP BY pickup_datetime, dropoff_datetime, trip_duration, trip_distance, total_amount;
''', con=conn)

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 1min 4s


In [4]:
df.to_csv('trips_New_York_Marriott_Marquis_to_jfk.csv', index=False)